In [6]:
%matplotlib inline
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import seaborn as sns
import json
import matplotlib.pyplot as plt
from gensim import corpora, models

In [137]:
def load_json_to_df(datapass):
    data = [] 
    with open(datapass) as data_file: 
        for f in data_file:
            data.append(json.loads(f))
    df = pd.DataFrame(data)
    return df

In [3]:
business = load_json_to_df("../../../dataset/business.json")

In [4]:
review = load_json_to_df("../../../dataset/review.json")

## Overview:
Train the matrix of business subscores by minimizing a loss function for each restaurant, defined by sum((rec - rating)^2), where rec is (rating subscores dot user preference).

We obtain user preference by running an LDA on all the text reviews, that is, we are setting the preference column as a constant in the loss function. 

The weight for each topic is calculated by normalizing the sum of the probabilty that each word of one user's texts occurs in the topics generated by LDA on all reviews.

In [132]:
review.head(5)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


## Preprocessing:
0. Extract all the restaurant reviews.
1. tokenize
2. stop words
3. stemmize
4. get only nouns and adjs

In [9]:
is_rest = []
for i in business['categories']:
    
    if 'Restaurants' in i or 'Food' in i:
        is_rest.append(True)
    else:
        is_rest.append(False)
restaurants = business.loc[is_rest]
restaurants.shape

(65028, 15)

In [10]:
rest_id = restaurants['business_id']
rest_review = review.loc[review['business_id'].isin(rest_id)]
rest_review.shape

(3216548, 9)

In [24]:
groupby_user = rest_review.groupby('user_id').size().reset_index(name='counts')

In [134]:
groupby_user.sort_values('counts',ascending=False)

,user_id,counts
186623,CxDOIDnH8gp9KXzpBHJYXw,2828
525915,bLbSNkLggFnqwNNzzq-Ijw,1473
191626,DK57YibC5ShBmqQl97CKog,1021
352049,PKEzKWv_FktMm2mGPjwd0Q,1001
555828,d_TBs6J3twMy9GChqUEXkg,991
539407,cMEtAiW60I5wE_vLfTxoJQ,934
205236,ELcQDlf69kb-ihJfxZyL0A,922
415516,U4INQZOPSUaj8hMjLlZ3KA,791
421891,UYcmGbelzRa0Q6JqzLoguw,790
365144,QJI9OSEn6ujRCtrX06vs1w,741


In [49]:
#preprocess
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from nltk.corpus import stopwords
en_stop = stopwords.words('english')

from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

def preprocess(text):
    raw = text.lower()
    tokens = tokenizer.tokenize(raw)
    
    pospeech=[]
    tag = nltk.pos_tag(tokens)
    for j in tag:
        if j[1] == 'NN' or j[1] == 'JJ':
            pospeech.append(j[0])
    # remove stop words from tokens
    stopped_tokens = [i for i in pospeech if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    return stemmed_tokens



## Generating LDA 
For demo purpose, running LDA on only one user.

In [261]:
NUM_TOPICS = 120
from gensim import corpora, models


def getlda(df):
    texts = []
    for i in df['text']:
        texts.append(preprocess(i))
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    # generate LDA model
    ldamodel = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word = dictionary, passes=20)
    return ldamodel, corpus

## Calculate preference maxtrix
Add up the probabity that each word in the corpus in each topic, then add up for each topic and normalize.

In [262]:
def getprefer(userid, ldamodel, df):
    user_reviews = df.loc[df['user_id'] == userid]
    l = np.zeros(NUM_TOPICS)
    texts = []
    for t in user_reviews['text']:
        texts.append(preprocess(t))
    
    for i in ldamodel.get_document_topics(corpus):
        for topic in i:
            l[topic[0]] += topic[1]
    topic_likelihood = []
    for i in l:
        topic_likelihood.append(i/sum(l))
    return topic_likelihood

## Minimize loss function
<img src="formula.png">
Since we setting the topics for each user as constant, the corpus likelihood here is a constant so we only need to deal with rating error.

We used Sequential Least SQuares Programming (SLSQP) to minize the function.

Intial guess was randomized between 1.0 and 5.0

Might need to explore bias and hyperparameters in the future.

In [263]:
from scipy.optimize import minimize
def min_loss(raw_prefer, actual_rating):
    dim = NUM_TOPICS
    prefer = []
    for i in raw_prefer:
        prefer.append(np.asarray(i, dtype=np.float32))
    prefer = np.asarray(prefer)
    print (prefer)
    print (actual_rating)
    bound = []
    for i in range(0,dim):
        bound.append((1.0,5.0))
    bnds = tuple(bound)


    def f(x,prefer,actual_rating):

        return sum(abs(np.dot(x,np.transpose(prefer)) - actual_rating))
    initial_guess = [np.random.uniform(1.0,5.0,dim)]
    #initial_guess = np.full(5,2.5)
    result = minimize(f, initial_guess, args=(prefer,actual_rating), method='SLSQP', bounds=bnds)
    if result.success:
        fitted_params = result.x
        print(fitted_params)
        print(result.fun)
        print(result.nit)
        return result
    else:
        raise ValueError(result.message)

## Combine everything
For each restaurant, run the algorithm to get the subscores, which will take a couple days on MacBook ~~Pro~~.

In [241]:
def add_prefer_to_df(df, lda, groupbyusers):
    d = {}
    new_df = df
    for u in groupbyusers['user_id']:
        d[u] = getprefer(u,lda,new_df)
    df['preference'] = df.apply(lambda x: d[x['user_id']],axis=1)
    return new_df

In [188]:
def train_rest_subscore(bizid, df):
    biz = df.loc[df['business_id'] == bizid]
    rating = biz['stars']
    preference = biz['preference']
    result = min_loss(preference.values, rating.values)
    return result.x

In [138]:
#sample = rest_review.sample(50000)

In [139]:
#sample_lda = getlda(sample)

In [142]:
#groupby_user_sample = sample.groupby('user_id').size().reset_index(name='counts')

In [242]:
#perfer_added_to_sample = add_prefer_to_df(sample, sample_lda, groupby_user_sample)

In [271]:
#perfer_added_to_sample.head(10)

In [250]:
#result_score = train_rest_subscore('HM0sWlwg3ONJ6syMSh_9zw', perfer_added_to_sample)

[[ 0.51775599  0.48224401  0.          0.          0.        ]
 [ 0.57377005  0.42622998  0.          0.          0.        ]
 [ 0.27269748  0.27813289  0.          0.44916964  0.        ]
 [ 0.24649563  0.26718768  0.          0.48631668  0.        ]
 [ 0.19245699  0.7673232   0.01333379  0.0135341   0.01335193]]
[5 1 3 3 1]
[ 1.22209286  1.          3.34416122  4.99997621  4.04593793]
4.324013475325012
14


In [245]:
#groupby_biz_sample = rest_review.groupby('business_id').size().reset_index(name='counts')

In [246]:
def add_subscore_to_df(df, groupbybiz):
    d = {}
    new_df = df
    for u in groupbybiz['business_id']:
        d[u] = train_rest_subscore(u, df)
    df['subscore'] = df.apply(lambda x: d[x['business_id']],axis=1)
    return new_df

In [276]:
lda = getlda(rest_review)

KeyboardInterrupt: 

In [ ]:
groupby_user = rest_review.groupby('user_id').size().reset_index(name='counts')

In [ ]:
perfer_added = add_prefer_to_df(rest_review, lda, groupby_user)

In [ ]:
subscore_added = add_subscore_to_df(prefer_added, lda, group)

## Future
Evaluating the performance.

Come up with a method that does not set the topics as a whole but one for each user. However, we are struggling to see how to match topics between user and restaurant without manual labelling.